In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import pandas as pd

In [ ]:


class CombinedModel(nn.Module):
    def __init__(self, transformer_output_dim, gcn_output_dim, combined_dim, output_dim):
        super(CombinedModel, self).__init__()
        self.transformer_output_dim = transformer_output_dim
        self.gcn_output_dim = gcn_output_dim
        self.combined_dim = combined_dim
        self.output_dim = output_dim

        self.ff1 = nn.Linear(transformer_output_dim + gcn_output_dim, combined_dim)
        self.activation = nn.ReLU()
        self.ff2 = nn.Linear(combined_dim, combined_dim)

        self.output_layer = nn.Linear(combined_dim, output_dim)

    def forward(self, transformer_output, gcn_output):
        # Reshape the inputs if necessary
        transformer_output = transformer_output.view(-1, self.transformer_output_dim)
        gcn_output = gcn_output.view(-1, self.gcn_output_dim)

        # Concatenate the representations
        combined_input = torch.cat([transformer_output, gcn_output], dim=1)

        # Apply feedforward layers
        combined_output = self.ff1(combined_input)
        combined_output = self.activation(combined_output)
        combined_output = self.ff2(combined_output)

        # Predict ged_sb
        ged_sb = self.output_layer(combined_output)

        return ged_sb

def load_data(data_path):
    combined_output = pd.read_csv(data_path)
    X_transformer = transformer_model
    X_gcn = gcn_model
    y = torch.tensor(combined_output['ged_sb'].values.tolist())
    return X_transformer, X_gcn, y

def train(model, dataloader, criterion, optimizer, epochs):
    for epoch in range(epochs):
        epoch_loss = 0.0
        for transformer_output, gcn_output, target in dataloader:
            # Zero the gradients
            optimizer.zero_grad()

            # Forward pass
            output = model(transformer_output, gcn_output)
            loss = criterion(output, target)

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

            # Update the epoch loss
            epoch_loss += loss.item()

        # Print the epoch loss
        print(f'Epoch {epoch+1}/{epochs}, Loss: {epoch_loss/len(dataloader)}')

    return model


In [ ]:
def main():
    # Load data
    X_transformer, X_gcn, y = load_data('combined_output.csv')

    # Create a TensorDataset and DataLoader
    dataset = TensorDataset(X_transformer, X_gcn, y)
    dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

    # Initialize the model
    model = CombinedModel(transformer_output_dim=X_transformer.shape[-1],
                          gcn_output_dim=X_gcn.shape[-1],
                          combined_dim=256,
                          output_dim=1)

    # Define the loss function and optimizer
    criterion = nn.MSELoss()
    optimizer = optim.RMSProp(model.parameters(), lr=0.001)

    # Train the model
    trained_model = train(model, dataloader, criterion, optimizer, epochs=100)

    # Save the trained model
    torch.save(trained_model.state_dict(), 'combined_model.pth')

if __name__ == '__main__':
    main()